In [2]:
# PDF 파일을 읽고 정규식 기준으로 청킹하여 JSONL로 저장하는 코드
import os
import re
import json
from typing import List, Dict, Any
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def custom_regex_split(text: str, pattern: str = r"^\<.+\>") -> List[str]:
    """
    정규식 패턴을 기준으로 텍스트를 청킹합니다.
    
    Args:
        text: 분할할 텍스트
        pattern: 청크 시작점을 나타내는 정규식 패턴
    
    Returns:
        청킹된 텍스트 리스트
    """
    # 줄별로 분할
    lines = text.split('\n')
    chunks = []
    current_chunk = []
    
    for line in lines:
        # 정규식 패턴에 매치되는 경우 새 청크 시작
        if re.match(pattern, line.strip()):
            # 이전 청크가 있으면 저장
            if current_chunk:
                chunks.append('\n'.join(current_chunk).strip())
            # 새 청크 시작
            current_chunk = [line]
        else:
            current_chunk.append(line)
    
    # 마지막 청크 추가
    if current_chunk:
        chunks.append('\n'.join(current_chunk).strip())
    
    return [chunk for chunk in chunks if chunk.strip()]  # 빈 청크 제거

def load_and_chunk_pdf(pdf_path: str) -> List[Dict[str, Any]]:
    """
    PDF를 로드하고 정규식 기준으로 청킹합니다.
    
    Args:
        pdf_path: PDF 파일 경로
    
    Returns:
        청킹된 문서들의 리스트
    """
    # 파일 존재 확인
    if not os.path.isfile(pdf_path):
        raise ValueError(f"파일이 존재하지 않습니다: {pdf_path}")
        
    # PDF 로드
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # 모든 페이지의 텍스트를 하나로 합침
    full_text = '\n'.join([doc.page_content for doc in documents])
    
    # 정규식 기준으로 청킹
    chunks = custom_regex_split(full_text)
    
    # 결과 포맷팅
    result = []
    for i, chunk in enumerate(chunks):
        if chunk.strip():  # 빈 청크 제외
            result.append({
                "id": f"chunk_{i+1}",
                "content": chunk,
                "source": pdf_path,
                "chunk_index": i + 1,
                "length": len(chunk)
            })
    
    return result

def save_to_jsonl(data: List[Dict[str, Any]], output_path: str):
    """
    데이터를 JSONL 형태로 저장합니다.
    
    Args:
        data: 저장할 데이터 리스트
        output_path: 출력 파일 경로
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

# 메인 실행 코드
if __name__ == "__main__":
    # 현재 디렉토리 확인
    current_dir = os.getcwd()
    
    # 절대 경로로 PDF 파일 경로 설정
    pdf_path = os.path.abspath(os.path.join(current_dir, "..", "resource", "rag_data.pdf"))
    output_path = "chunks.jsonl"
    
    # 파일 존재 확인
    if not os.path.exists(pdf_path):
        print(f"파일을 찾을 수 없습니다: {pdf_path}")
        # 대안 경로들 시도
        alternative_paths = [
            os.path.join(current_dir, "resource", "rag_data.pdf"),
            "/resource/국어 지식 기반 생성(RAG) 참조 문서.pdf"
        ]
        
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                pdf_path = alt_path
                print(f"파일을 찾았습니다: {pdf_path}")
                break
        else:
            print("모든 경로에서 파일을 찾을 수 없습니다.")
            print(f"현재 디렉토리: {current_dir}")
            exit(1)
    
    print(f"PDF 파일 경로: {pdf_path}")
    print("PDF 파일을 로드하고 청킹을 시작합니다...")
    
    # PDF 로드 및 청킹
    chunks = load_and_chunk_pdf(pdf_path)
    
    print(f"총 {len(chunks)}개의 청크가 생성되었습니다.")
    
    # JSONL로 저장
    save_to_jsonl(chunks, output_path)
    
    print(f"청크들이 {output_path}에 저장되었습니다.")
    
    # 첫 번째 청크 미리보기
    if chunks:
        print(f"\n첫 번째 청크 미리보기:")
        print(f"ID: {chunks[0]['id']}")
        print(f"길이: {chunks[0]['length']} 문자")
        print(f"내용 (처음 200자):")
        print(chunks[0]['content'][:200] + "..." if len(chunks[0]['content']) > 200 else chunks[0]['content'])


PDF 파일 경로: /home/hdd340/Korean_QA_RAG_2025/resource/rag_data.pdf
PDF 파일을 로드하고 청킹을 시작합니다...
총 115개의 청크가 생성되었습니다.
청크들이 chunks.jsonl에 저장되었습니다.

첫 번째 청크 미리보기:
ID: chunk_1
길이: 45 문자
내용 (처음 200자):
<띄어쓰기 - 한글 맞춤법 제2항> 
문장의 각 단어는 띄어 씀을 원칙으로 한다.
